# Welcome to Miniverse

**Build emergent agent-based simulations powered by LLMs.**

This notebook shows you what Miniverse does through a simple Mars habitat simulation.

## What you'll see:

- **3 agents** (Commander, Engineer, Scientist) coordinate on Mars base operations
- **LLM-driven decisions** - agents plan, communicate, and adapt
- **Emergent behavior** - watch coordination emerge from individual goals
- **Full observability** - inspect plans, memories, and reasoning

**Setup required:**
```bash
export LLM_PROVIDER=openai
export LLM_MODEL=gpt-4
export OPENAI_API_KEY=your_key
```

Let's run it!

In [ ]:
import os
import asyncio
from datetime import datetime, timezone

# Check LLM config
provider = os.getenv('LLM_PROVIDER')
model = os.getenv('LLM_MODEL')

if not provider or not model:
    print('❌ LLM not configured. Set LLM_PROVIDER, LLM_MODEL, and API key.')
else:
    print(f'✓ Using {provider}/{model}')

In [ ]:
from miniverse import (
    Orchestrator, AgentProfile, AgentStatus, WorldState,
    ResourceState, EnvironmentState, SimulationRules,
    Stat, AgentAction
)
from miniverse.cognition import AgentCognition, LLMExecutor, LLMPlanner, Scratchpad

# Mars habitat physics
class MarsHabitatRules(SimulationRules):
    """Simple Mars base: manage power, conduct research, handle maintenance."""
    
    def apply_tick(self, state, tick):
        updated = state.model_copy(deep=True)
        
        # Get resources
        power = updated.resources.get_metric('power', default=100, unit='kWh')
        research = updated.resources.get_metric('research_progress', default=0, unit='%')
        maintenance = updated.resources.get_metric('system_health', default=100, unit='%')
        
        # Process agent activities
        for agent in updated.agents:
            if agent.activity == 'generate_power':
                power.value = min(100, float(power.value) + 15)
            elif agent.activity == 'research':
                research.value = min(100, float(research.value) + 10)
                power.value = max(0, float(power.value) - 8)  # Research uses power
            elif agent.activity == 'maintenance':
                maintenance.value = min(100, float(maintenance.value) + 12)
        
        # Natural degradation
        power.value = max(0, float(power.value) - 5)  # Base consumption
        maintenance.value = max(0, float(maintenance.value) - 3)  # Systems degrade
        
        updated.tick = tick
        return updated
    
    def validate_action(self, action, state):
        return True  # Accept all actions

print('Mars habitat physics defined ✓')

In [ ]:
# Initial world state
world_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(metrics={}),
    resources=ResourceState(metrics={
        'power': Stat(value=60, unit='kWh', label='Power Reserve'),
        'research_progress': Stat(value=25, unit='%', label='Research Progress'),
        'system_health': Stat(value=70, unit='%', label='System Health')
    }),
    agents=[
        AgentStatus(agent_id='commander', display_name='Commander Liu'),
        AgentStatus(agent_id='engineer', display_name='Engineer Kim'),
        AgentStatus(agent_id='scientist', display_name='Dr. Patel')
    ]
)

# Agent profiles
agents = {
    'commander': AgentProfile(
        agent_id='commander',
        name='Commander Liu',
        age=42,
        background='15 years military + 5 years NASA mission experience',
        role='mission_commander',
        personality='strategic, calm under pressure, prioritizes crew safety',
        skills={'leadership': 'expert', 'decision_making': 'expert'},
        goals=['Ensure crew safety', 'Complete mission objectives', 'Maintain team morale'],
        relationships={'engineer': 'trusts technical judgment', 'scientist': 'values research input'}
    ),
    'engineer': AgentProfile(
        agent_id='engineer',
        name='Engineer Kim',
        age=35,
        background='Aerospace engineer, 8 years ISS systems experience',
        role='systems_engineer',
        personality='detail-oriented, proactive, safety-focused',
        skills={'systems': 'expert', 'power_management': 'expert', 'maintenance': 'expert'},
        goals=['Keep systems operational', 'Optimize power usage', 'Prevent failures'],
        relationships={'commander': 'reports to', 'scientist': 'coordinates with'}
    ),
    'scientist': AgentProfile(
        agent_id='scientist',
        name='Dr. Patel',
        age=38,
        background='Astrobiologist, 10 years Mars research',
        role='lead_scientist',
        personality='curious, methodical, mission-driven',
        skills={'research': 'expert', 'analysis': 'expert'},
        goals=['Advance Mars research', 'Make discoveries', 'Publish findings'],
        relationships={'commander': 'collaborates with', 'engineer': 'relies on for equipment'}
    )
}

print(f'Created 3 agents: {list(agents.keys())}')

In [ ]:
# Agent cognition: LLM executor + planner
cognition_map = {
    'commander': AgentCognition(
        executor=LLMExecutor(),
        planner=LLMPlanner(),
        scratchpad=Scratchpad()
    ),
    'engineer': AgentCognition(
        executor=LLMExecutor(),
        planner=LLMPlanner(),
        scratchpad=Scratchpad()
    ),
    'scientist': AgentCognition(
        executor=LLMExecutor(),
        planner=LLMPlanner(),
        scratchpad=Scratchpad()
    )
}

# Agent prompts (role instructions)
agent_prompts = {
    'commander': '''You are Commander Liu on a Mars habitat.

Your responsibilities:
- Oversee base operations and crew coordination
- Make strategic decisions about priorities
- Communicate with team via brief updates

Available actions: generate_power, research, maintenance, coordinate, rest

Current priorities: Safety > Systems > Research''',
    
    'engineer': '''You are Engineer Kim maintaining Mars habitat systems.

Your responsibilities:
- Monitor power levels and system health
- Perform maintenance when needed
- Generate power when reserves are low
- Advise commander on technical issues

Available actions: generate_power, maintenance, rest

Keep power above 40 kWh and systems above 60%.''',
    
    'scientist': '''You are Dr. Patel, lead scientist on Mars.

Your responsibilities:
- Conduct research experiments
- Balance research with base needs
- Communicate findings and coordinate with crew

Available actions: research, rest

Research is important but requires power - coordinate with engineer.'''
}

print('Agent cognition configured with LLM executor + planner ✓')

In [ ]:
# Create orchestrator
orchestrator = Orchestrator(
    world_state=world_state,
    agents=agents,
    world_prompt='',
    agent_prompts=agent_prompts,
    simulation_rules=MarsHabitatRules(),
    agent_cognition=cognition_map,
    llm_provider=provider,
    llm_model=model
)

print('🚀 Running 5 ticks on Mars...')
print('Watch agents coordinate to manage power, research, and maintenance\n')

result = await orchestrator.run(num_ticks=5)

print('\n✓ Simulation complete!')

In [ ]:
# Inspect final state
final = result['final_state']

print('=== FINAL STATE ===')
print(f'Tick: {final.tick}')
print(f'\nResources:')
for key, stat in final.resources.metrics.items():
    print(f'  {stat.label}: {stat.value}{stat.unit or ""}')

print(f'\nAgent Activities:')
for agent in final.agents:
    print(f'  {agent.display_name}: {agent.activity}')

## 🔍 What Just Happened?

Let's peek under the hood to see how agents made decisions.

In [ ]:
# Get actions from last tick
sim_id = result['simulation_id']
tick_5_actions = await orchestrator.persistence.get_actions(sim_id, tick=5)

print('=== TICK 5 AGENT REASONING ===')
for action in tick_5_actions:
    agent_name = agents[action.agent_id].name
    print(f'\n{agent_name}:')
    print(f'  Action: {action.action_type}')
    print(f'  Reasoning: {action.reasoning}')
    if action.communication:
        print(f'  Communication: "{action.communication}"')

In [ ]:
# Check engineer's memories (saw power crisis?)
engineer_memories = await orchestrator.persistence.get_memories(sim_id, agent_id='engineer')

print(f'=== ENGINEER\'S MEMORIES ({len(engineer_memories)} total) ===')
for i, mem in enumerate(engineer_memories[-3:], 1):  # Last 3
    print(f'\n{i}. Tick {mem.tick}:')
    print(f'   {mem.content[:100]}...' if len(mem.content) > 100 else f'   {mem.content}')

## 💡 What You Just Saw

**Miniverse simulated 5 hours on Mars with 3 LLM-powered agents:**

### Each Tick:
1. **Physics applied** - power drained, systems degraded, actions took effect
2. **Agents observed** - saw resource levels, other agents' activities
3. **Agents planned** - LLM generated multi-step plans based on role and situation
4. **Agents acted** - chose actions aligned with plans and goals
5. **Memories stored** - observations saved for future reasoning

### Key Features:
- **Emergent coordination** - no hardcoded teamwork, agents adapted based on situation
- **LLM reasoning** - decisions explained in natural language
- **Memory persistence** - agents remember past events
- **Full observability** - inspect plans, memories, reasoning at any point

### The Setup (20 lines):
```python
# 1. Define physics (SimulationRules subclass)
class MarsHabitatRules(SimulationRules):
    def apply_tick(self, state, tick):
        # Your domain logic here
        return updated_state

# 2. Create agents with profiles
agents = {"commander": AgentProfile(...), ...}

# 3. Configure cognition (LLM-powered)
cognition = {"commander": AgentCognition(
    executor=LLMExecutor(),
    planner=LLMPlanner()
)}

# 4. Run simulation
orchestrator = Orchestrator(...)
result = await orchestrator.run(num_ticks=5)
```


## 🚀 Next Steps

**Explore more:**
- **`overview.ipynb`** - Deep dive into core concepts
- **`examples/workshop/`** - Progressive examples (deterministic → LLM → multi-agent)
- **Documentation** - Architecture, API reference, patterns

**Build your own:**
1. Define your domain's physics (SimulationRules)
2. Create agent profiles (roles, goals, relationships)
3. Choose cognition modules (deterministic, LLM, or hybrid)
4. Run and observe emergent behavior

**Common use cases:**
- Social simulations (Stanford Generative Agents style)
- Multi-agent systems research
- Game AI and NPC behavior
- Organizational dynamics
- Economic modeling

Miniverse gives you:
- ✓ Deterministic physics (controllable rules)
- ✓ Emergent cognition (LLM intelligence)
- ✓ Full observability (inspect everything)
- ✓ Modular design (swap strategies easily)
